In [267]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from IPython.display import Markdown
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, OrdinalEncoder
from imblearn.over_sampling import RandomOverSampler, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers


raw_data = pd.read_csv('customer.csv')
masked_df = pd.read_csv('customer_test_masked.csv')

In [268]:
label_features = ['state_of_res']
categorical_features = ['sex','housing_type', 'gas_category','is_employed']
numerical_features = ['income', 'num_vehicles', 'age', 'gas_usage', 'rooms', 'age_income','age_vehicles','income_gas']
target_feature = 'health_ins'
id_feature = 'custid'

# Define a preprocessor globally so it can be reused
preprocessor = ColumnTransformer(
    transformers=[
        ('label', OrdinalEncoder(), label_features),
        ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features),
        ('scaler', StandardScaler(), numerical_features)
    ],
    remainder='passthrough'
)

def preprocess_dataframe(df, fit_preprocessor=True):
    global preprocessor  # Use the same preprocessor instance
    
    # Drop the target feature for feature processing
    df_features = df.drop(target_feature, axis=1, errors='ignore')
    
    if fit_preprocessor:
        # Fit and transform the preprocessor on training data
        features_processed = preprocessor.fit_transform(df_features)
    else:
        # Only transform using an already fitted preprocessor
        features_processed = preprocessor.transform(df_features)
    
    # Extract feature names from the preprocessor
    feature_names = (
        label_features +
        preprocessor.named_transformers_['onehot'].get_feature_names_out(categorical_features).tolist() +
        numerical_features +
        [id_feature]
    )
    
    # Create a DataFrame for the processed features
    df_processed = pd.DataFrame(features_processed, columns=feature_names)
    
    # Ensure indices align
    df_processed = df_processed.reset_index(drop=True)
    df_processed['income_state'] = df_processed['income'] * df_processed['state_of_res']
    # Add the target feature back if it exists in the original DataFrame
    if target_feature in df.columns:
        df_target = df[target_feature].reset_index(drop=True)
        df_processed[target_feature] = df_target
    
    return df_processed

In [269]:
def smote_data(X, y):
    smote = ADASYN(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)
    return X_resampled, y_resampled

In [270]:
def classify_sample_then_split(df, sample_function, params, classifier):
    # Preprocess data
    preprocessed_df = preprocess_dataframe(df)

    X = preprocessed_df.drop(target_feature, axis=1)  
    y = preprocessed_df[target_feature]
        
    # Sample data
    X_resampled, y_resampled = sample_function(X, y)
    
    # train test split
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)
    
    # Train model with GridSearchCV
    grid_search = GridSearchCV(estimator=classifier, param_grid=params, cv=5)
    
    grid_search.fit(X_train, y_train)
    
    # Predict
    y_pred = grid_search.predict(X_test)
    
    # Evaluate
    print(f'F1 Score: {f1_score(y_test, y_pred)}')
    print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
    print(f'Best Parameters: {grid_search.best_params_}')
    print(classification_report(y_test, y_pred))

In [271]:
def handle_gas_feature(df):
    
    df_copy = df.copy()
    
    gas_median = df_copy[df_copy['gas_usage'] > 3].gas_usage.median()

    non_bill_cases_values = {
        1: gas_median,
        2: gas_median,
        3: 0
    }
    non_bill_cases = {
        1: 'Included in rent', 
        2: 'Included in electricity', 
        3: 'No charge'
    }

    # New feature for gas usage category
    df_copy['gas_category'] = df_copy['gas_usage'].replace(non_bill_cases).where(
        df_copy['gas_usage'].isin(non_bill_cases.keys()), 'Actual Bill'
    )
    df_copy.loc[pd.isna(df_copy['gas_usage']), 'gas_category'] = 'Unknown'

    # Replace non-bill cases' values or maintain if not included
    df_copy['gas_usage'] = df_copy['gas_usage'].replace(non_bill_cases_values)
    df_copy['gas_usage'] = df_copy['gas_usage'].fillna(gas_median)
    df_copy['income_gas'] = df_copy['income'] * df_copy['gas_usage']
    
    return df_copy

In [272]:
def apply_data_engineering(df):
    df_filtered = df.copy()
    
    df_filtered.drop(columns=['Unnamed: 0', 'code_column','marital_status','recent_move_b'], inplace=True)

    rows_with_missing = df_filtered.isnull().sum(axis=1) > 2
    df_filtered.drop(df_filtered[rows_with_missing].index, inplace=True)
    
    df_filtered['is_employed'] = df_filtered['is_employed'].fillna(False)
    
    df_filtered['num_vehicles'] = df_filtered['num_vehicles'].fillna(round(df['num_vehicles'].mean(),0))
    
    # df_filtered['recent_move_b'] = df_filtered['recent_move_b'].fillna(df_filtered['recent_move_b'].mode()[0])
    # df_filtered['recent_move_b'] = df_filtered['recent_move_b'].replace({'T': True, 'F': False})
    
    df_filtered['age'] = df_filtered['age'].replace(0, df_filtered['age'].median())
    rows_with_age_120 = df_filtered['age'] == 120
    df_filtered = df_filtered.drop(df_filtered[rows_with_age_120].index)
    
    df_filtered = handle_gas_feature(df_filtered)
    
    df_filtered['age_income'] = df_filtered.age * df_filtered.income
    df_filtered['age_vehicles'] = df_filtered.age * df_filtered.num_vehicles
    
    df_missing = df[rows_with_missing | rows_with_age_120]
    df_missing.drop(columns=['Unnamed: 0', 'code_column','marital_status','recent_move_b'], inplace=True)
    return df_filtered, df_missing

In [273]:
def apply_data_engineering_replace(df):
    df_replaced = df.copy()
    
    df_replaced.drop(columns=['Unnamed: 0', 'code_column', 'marital_status', 'recent_move_b'], inplace=True, errors='ignore')

    df_replaced['housing_type'] = df_replaced['housing_type'].fillna(df_replaced['housing_type'].mode()[0])
    df_replaced['is_employed'] = df_replaced['is_employed'].fillna(False)
    df_replaced['num_vehicles'] = df_replaced['num_vehicles'].fillna(round(df['num_vehicles'].mean(), 0))
    df_replaced['age'] = df_replaced['age'].replace(0, df_replaced['age'].median())
    df_replaced['age'] = df_replaced['age'].replace(120, df_replaced['age'].median())
    df_replaced['age'] = df_replaced['age'].fillna(df_replaced['age'].median())

    df_replaced = handle_gas_feature(df_replaced)

    df_replaced['age_income'] = df_replaced.age * df_replaced.income
    df_replaced['age_vehicles'] = df_replaced.age * df_replaced.num_vehicles

    return df_replaced


# SUBMISSION

## XGBoost

In [274]:
df_filtered, df_missing = apply_data_engineering(raw_data)
sub_filtered, sub_missing = apply_data_engineering(masked_df)

C:\Users\User\AppData\Local\Temp\ipykernel_11720\2130856372.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_filtered['is_employed'] = df_filtered['is_employed'].fillna(False)
C:\Users\User\AppData\Local\Temp\ipykernel_11720\2130856372.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_missing.drop(columns=['Unnamed: 0', 'code_column','marital_status','recent_move_b'], inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_11720\2130856372.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call 

In [275]:
def convert_feature_types(df):
    for col in df.select_dtypes(include=['object']).columns:
        if col in categorical_features or col in label_features:
            df[col] = df[col].astype('category')
        elif col in numerical_features:
            df[col] = df[col].astype('float')

In [276]:
xgb_train_df = pd.concat([df_filtered, df_missing], axis=0, ignore_index=True)

xgb_train_df.drop(columns=['custid'], inplace=True)

convert_feature_types(xgb_train_df)

In [277]:
#### THIS CODE IS FOR THE XGBOOST MODEL ####
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming df is your DataFrame and 'target' is the column you want to predict
X = xgb_train_df.drop(target_feature, axis=1)
y = xgb_train_df[target_feature]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

params = {
    'objective': 'binary:logistic',  # For binary classification
    'max_depth': 6,
    'eta': 0.3,
    'eval_metric': 'logloss'
}

num_rounds = 100
bst = xgb.train(params, dtrain, num_rounds)

preds = bst.predict(dtest)
predictions = [1 if pred > 0.5 else 0 for pred in preds]

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")


Accuracy: 0.9044070291655166


In [278]:
X_train = xgb_train_df.drop(columns=target_feature)
y_train = xgb_train_df[target_feature]

masked_df = pd.concat([sub_filtered, sub_missing], axis=0, ignore_index=True)

X_submission = masked_df.drop(columns=['custid', 'health_ins'])

convert_feature_types(X_submission)

X_sub_matrix = xgb.DMatrix(X_submission, enable_categorical=True)

y_pred = bst.predict(X_sub_matrix)

# Create a DataFrame for the predictions and the custid
xgb_submission_df = pd.DataFrame({
    'custid': masked_df['custid'],  # Retrieve the 'custid' column from the original DataFrame
    'health_ins': y_pred           # Add the predicted values
})

# Convert the numeric values in 'predicted_y' to 'TRUE' or 'FALSE' based on the median threshold
xgb_submission_df['health_ins'] = xgb_submission_df['health_ins'].apply(lambda x: 'TRUE' if x > 0.9 else 'FALSE')

# Store the DataFrame without column names
xgb_submission_df.to_csv('xgb_submission.csv', index=False)


## Random Forest

In [279]:
rf_train_data = apply_data_engineering_replace(raw_data)
rf_submission_data = apply_data_engineering_replace(masked_df)

rf_train_data = preprocess_dataframe(rf_train_data, fit_preprocessor=True)
rf_submission_data = preprocess_dataframe(rf_submission_data, fit_preprocessor=False)
rf_submission_data['health_ins'] = rf_submission_data['health_ins'].astype(bool)

C:\Users\User\AppData\Local\Temp\ipykernel_11720\3751234964.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced['is_employed'] = df_replaced['is_employed'].fillna(False)
C:\Users\User\AppData\Local\Temp\ipykernel_11720\3751234964.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced['is_employed'] = df_replaced['is_employed'].fillna(False)


In [280]:
# separate features
X = rf_train_data.drop([target_feature, id_feature], axis=1)
y = rf_train_data[target_feature]

# define model random forest model following the best parameters {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 20, 'n_estimators': 300}
rf_model = RandomForestClassifier(class_weight='balanced', criterion='gini', max_depth=20, n_estimators=300,max_features=10, min_samples_leaf=1, random_state=42)

# resample data
X_resampled, y_resampled = smote_data(X, y)

# fit model
rf_model.fit(X_resampled, y_resampled)

rf_submission_features = rf_submission_data.drop(columns=[target_feature, id_feature])

# predict
y_pred = rf_model.predict(rf_submission_features)

# Create a DataFrame for the predictions and the custid
rf_submission_df = pd.DataFrame({
    'custid': rf_submission_data['custid'],  # Retrieve the 'custid' column from the original DataFrame
    'health_ins': y_pred           # Add the predicted values
})

# Store the DataFrame without column names
rf_submission_df.to_csv('rf_submission.csv', index=False)



In [281]:
# BALANCED PREDICTIONS (50-50 split)

# Predict probabilities (if available) for more nuanced control
y_proba = rf_model.predict_proba(rf_submission_features)[:, 1]  # Probabilities for the positive class (True)

# Sort indices by predicted probability for a deterministic split
sorted_indices = np.argsort(y_proba)

# Determine the split point
half_count = len(y_proba) // 2

# Initialize an array of predictions
y_pred_balanced = np.zeros_like(y_proba, dtype=bool)

# Set the top half to True and the bottom half to False
y_pred_balanced[sorted_indices[:half_count]] = False
y_pred_balanced[sorted_indices[half_count:]] = True

# Create a DataFrame for the balanced predictions
rf_submission_balanced_df = pd.DataFrame({
    'custid': rf_submission_data['custid'],  # Retrieve the 'custid' column
    'health_ins': y_pred_balanced             # Balanced predictions
})

# Store the DataFrame without column names
rf_submission_balanced_df.to_csv('rf_submission_balanced.csv', index=False)


## Decision Tree

In [282]:
dt_train_data = apply_data_engineering_replace(raw_data)
dt_submission_data = apply_data_engineering_replace(masked_df)

dt_train_data = preprocess_dataframe(dt_train_data, fit_preprocessor=True)
dt_submission_data = preprocess_dataframe(dt_submission_data, fit_preprocessor=False)
dt_submission_data['health_ins'] = dt_submission_data['health_ins'].astype(bool)

C:\Users\User\AppData\Local\Temp\ipykernel_11720\3751234964.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced['is_employed'] = df_replaced['is_employed'].fillna(False)
C:\Users\User\AppData\Local\Temp\ipykernel_11720\3751234964.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced['is_employed'] = df_replaced['is_employed'].fillna(False)


In [283]:
# separate features
X = dt_train_data.drop([target_feature, id_feature], axis=1)
y = dt_train_data[target_feature]

# define model random forest model following the best parameters {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 29, 'splitter': 'best'}
dt_model = DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=29, splitter='best')

# resample data
X_resampled, y_resampled = smote_data(X, y)

# fit model
dt_model.fit(X_resampled, y_resampled)

dt_submission_features = dt_submission_data.drop(columns=[target_feature, id_feature])

# predict
y_pred = dt_model.predict(dt_submission_features)

# Create a DataFrame for the predictions and the custid
dt_submission_df = pd.DataFrame({
    'custid': dt_submission_data['custid'],  # Retrieve the 'custid' column from the original DataFrame
    'health_ins': y_pred           # Add the predicted values
})

# Store the DataFrame without column names
dt_submission_df.to_csv('dt_submission.csv', index=False)

In [284]:
# BALANCED PREDICTIONS (50-50 split)

# Predict probabilities (if available) for more nuanced control
y_proba = dt_model.predict_proba(dt_submission_features)[:, 1]  # Probabilities for the positive class (True)

# Sort indices by predicted probability for a deterministic split
sorted_indices = np.argsort(y_proba)

# Determine the split point
half_count = len(y_proba) // 2

# Initialize an array of predictions
y_pred_balanced = np.zeros_like(y_proba, dtype=bool)

# Set the top half to True and the bottom half to False
y_pred_balanced[sorted_indices[:half_count]] = False
y_pred_balanced[sorted_indices[half_count:]] = True

# Create a DataFrame for the balanced predictions
dt_submission_balanced_df = pd.DataFrame({
    'custid': dt_submission_data['custid'],  # Retrieve the 'custid' column
    'health_ins': y_pred_balanced             # Balanced predictions
})

# Store the DataFrame without column names
dt_submission_balanced_df.to_csv('dt_submission_balanced.csv', index=False)

# NEURAL NETWORK

In [285]:
nn_train_data = apply_data_engineering_replace(raw_data)
dt_submission_data = apply_data_engineering_replace(masked_df)

nn_train_data = preprocess_dataframe(nn_train_data, fit_preprocessor=True)
nn_submission_data = preprocess_dataframe(dt_submission_data, fit_preprocessor=False)
nn_submission_data['health_ins'] = nn_submission_data['health_ins'].astype(bool)

C:\Users\User\AppData\Local\Temp\ipykernel_11720\3751234964.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced['is_employed'] = df_replaced['is_employed'].fillna(False)
C:\Users\User\AppData\Local\Temp\ipykernel_11720\3751234964.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced['is_employed'] = df_replaced['is_employed'].fillna(False)


In [288]:
def create_nn_model():
    nn = Sequential()
    nn.add(layers.Dense(128, activation='relu', input_dim=19)) 
    nn.add(layers.Dense(64, activation='relu'))
    nn.add(layers.Dense(32, activation='relu'))
    nn.add(layers.Dense(1, activation='sigmoid')) 

    nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return nn

nn = create_nn_model()

X_train = nn_train_data.drop([target_feature, id_feature], axis=1)
y_train = nn_train_data[target_feature]

X_submission = nn_submission_data.drop(columns=[target_feature, id_feature])

X_respample, Y_resample = smote_data(X_train, y_train)

# Convert the data to float32
X_respample = X_respample.astype('float32')
Y_resample = Y_resample.astype('float32')
X_submission = X_submission.astype('float32')

# Train the model
nn.fit(X_respample, Y_resample, epochs=30, batch_size=32)

y_pred = nn.predict(X_submission)

y_pred = (y_pred > 0.6).astype(int)

# Create a DataFrame for the predictions and the custid
submission_df = pd.DataFrame({
    'custid': dt_submission_data['custid'],  # Retrieve the 'custid' column from the original DataFrame
    'health_ins': y_pred.flatten()        # Add the predicted values
})

# Convert the numeric values in 'predicted_y' to 'TRUE' and 'FALSE'
submission_df['health_ins'] = submission_df['health_ins'].map({1: 'TRUE', 0: 'FALSE'})

# Store the DataFrame without column names

submission_df.to_csv('submission_nn.csv', index=False)

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
4128/4128 ━━━━━━━━━━━━━━━━━━━━ 4s 712us/step - accuracy: 0.7112 - loss: 0.5690
Epoch 2/30
4128/4128 ━━━━━━━━━━━━━━━━━━━━ 3s 687us/step - accuracy: 0.7463 - loss: 0.5121
Epoch 3/30
4128/4128 ━━━━━━━━━━━━━━━━━━━━ 3s 654us/step - accuracy: 0.7639 - loss: 0.4831
Epoch 4/30
4128/4128 ━━━━━━━━━━━━━━━━━━━━ 3s 636us/step - accuracy: 0.7894 - loss: 0.4356
Epoch 5/30
4128/4128 ━━━━━━━━━━━━━━━━━━━━ 3s 642us/step - accuracy: 0.8094 - loss: 0.3987
Epoch 6/30
4128/4128 ━━━━━━━━━━━━━━━━━━━━ 3s 702us/step - accuracy: 0.8178 - loss: 0.3778
Epoch 7/30
4128/4128 ━━━━━━━━━━━━━━━━━━━━ 3s 685us/step - accuracy: 0.8259 - loss: 0.3603
Epoch 8/30
4128/4128 ━━━━━━━━━━━━━━━━━━━━ 3s 658us/step - accuracy: 0.8307 - loss: 0.3504
Epoch 9/30
4128/4128 ━━━━━━━━━━━━━━━━━━━━ 3s 611us/step - accuracy: 0.8373 - loss: 0.3422
Epoch 10/30
4128/4128 ━━━━━━━━━━━━━━━━━━━━ 3s 623us/step - accuracy: 0.8411 - loss: 0.3330
Epoch 11/30
4128/4128 ━━━━━━━━━━━━━━━━━━━━ 3s 611us/step - accuracy: 0.8440 - loss: 0.3269
Epoch 12

# KNN

In [289]:
knn_train_data = apply_data_engineering_replace(raw_data)
knn_submission_data = apply_data_engineering_replace(masked_df)

knn_train_data = preprocess_dataframe(knn_train_data, fit_preprocessor=True)
knn_submission_data = preprocess_dataframe(knn_submission_data, fit_preprocessor=False)
knn_submission_data['health_ins'] = knn_submission_data['health_ins'].astype(bool)

C:\Users\User\AppData\Local\Temp\ipykernel_11720\3751234964.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced['is_employed'] = df_replaced['is_employed'].fillna(False)
C:\Users\User\AppData\Local\Temp\ipykernel_11720\3751234964.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced['is_employed'] = df_replaced['is_employed'].fillna(False)


In [290]:
knn = KNeighborsClassifier(n_neighbors=1)

X_train = knn_train_data.drop([target_feature, id_feature], axis=1)

y_train = knn_train_data[target_feature]

X_submission = knn_submission_data.drop(columns=[target_feature, id_feature])

X_resampled, y_resampled = smote_data(X_train, y_train)

knn.fit(X_resampled, y_resampled)

y_pred = knn.predict(X_submission)

# Create a DataFrame for the predictions and the custid

knn_submission_df = pd.DataFrame({
    'custid': knn_submission_data['custid'],  # Retrieve the 'custid' column from the original DataFrame
    'health_ins': y_pred           # Add the predicted values
})

# Store the DataFrame without column names
knn_submission_df.to_csv('knn_submission.csv', index=False)